In [ ]:
from tensorflow.keras import datasets, layers, models, callbacks
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import path
import os
from tqdm import tqdm
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

## Loading Dataset

In [ ]:
X_train = image_dataset_from_directory('/content/normalizedImages/train', image_size=(384, 384), batch_size=8, subset="training", validation_split=0.1, seed=11, label_mode='categorical', class_names=['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'LAYING', 'SITTING', 'STANDING'])

In [ ]:
X_val = image_dataset_from_directory('/content/normalizedImages/train', image_size=(384, 384), batch_size=8, subset="validation", validation_split=0.1, seed=11, label_mode='categorical', class_names=['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'LAYING', 'SITTING', 'STANDING'])

In [ ]:
X_test = image_dataset_from_directory('/content/normalizedImages/test', image_size=(384, 384), batch_size=8, label_mode='categorical', class_names=['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'LAYING', 'SITTING', 'STANDING'])

## Building Model

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(384, 384, 3)))
    
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(384, 384, 3)))
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    
    # model.add(layers.Dropout(0.5))
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))

    model.compile(optimizer="adam", loss="kullback_leibler_divergence", metrics=["accuracy"])
    print(model.summary())
    return model

In [ ]:
model = build_model()

In [ ]:
es = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3)
model.fit(X_train, epochs=15, verbose=True, batch_size=8, callbacks=[es], validation_data=X_val, class_weight={0: 1, 1: 1, 2:1, 3:1, 4:1.4, 5:1})

In [ ]:
model.evaluate(X_test)

## Confusion Matrix

In [ ]:
y_true = []
y_pred = []

for image_batch, label_batch in X_test:   # use dataset.unbatch() with repeat
   # append true labels
   y_true.append(label_batch)
   # compute predictions
   preds = model.predict(image_batch)
   # append predicted labels
   y_pred.append(np.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)
rounded_labels=np.argmax(correct_labels, axis=1)

In [ ]:
confusion_matrix(predicted_labels, rounded_labels)